In [18]:
#QUESTION1
import requests
import zipfile
import io
import os

# URL for the Dakshina dataset
url = "https://github.com/google-research-datasets/dakshina/archive/refs/heads/master.zip"

# Directory to save the dataset
dataset_dir = "dakshina_dataset"

# Create the directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)

# Download the dataset
response = requests.get(url)

# Check if the download was successful
if response.status_code == 200:
    # Extract the dataset
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dataset_dir)
    print(f"Dakshina dataset downloaded and extracted to '{dataset_dir}'")
else:
    print(f"Failed to download the dataset: {response.status_code} {response.reason}")


Dakshina dataset downloaded and extracted to 'dakshina_dataset'


In [19]:

from google.colab import files

# Upload your dataset
uploaded = files.upload()


Saving hi.translit.sampled.train.tsv to hi.translit.sampled.train (1).tsv


In [20]:
import os

# List all files in the current directory (content)
print(os.listdir("/content"))

['.config', 'hi.translit.sampled.train.tsv', 'dakshina_dataset', 'hi.translit.sampled.train (1).tsv', 's2s_model.keras', 'sample_data']


In [21]:
import pandas as pd

# Define the path to the uploaded file
lexicon_path = "/content/hi.translit.sampled.train.tsv"

# Load the file into a DataFrame
df = pd.read_csv(lexicon_path, sep='\t', header=None, names=['native', 'latin', 'attested'])

# Show the first 5 rows
print(df.head())


    native     latin  attested
0       अं        an         3
1  अंकगणित  ankganit         3
2     अंकल     uncle         4
3    अंकुर     ankur         4
4   अंकुरण   ankuran         3


In [22]:
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Parameters
batch_size = 64
epochs = 180
latent_dim = 256
num_samples = 50000  # Limit the number of training samples

# Path to the Dakshina dataset (adjust if you have different path)
data_path = "/content/hi.translit.sampled.train.tsv"

# Step 1: Load the dataset
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    native, latin, attested = line.split("\t")
    input_text = latin.strip()
    target_text = "\t" + native.strip() + "\n"  # Start and end tokens

    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

# Add space character to both input and target characters if not already included
input_characters.add(" ")
target_characters.add(" ")

# Rebuild the input_token_index and target_token_index after adding space
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Step 2: Create empty arrays to store one-hot encodings
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# Step 3: Fill in the one-hot encoded arrays
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0  # Fix added here
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Define the sequence-to-sequence model
# Step 4: Define Encoder and Decoder Model
# Encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Step 5: Compile and train the model
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Save the model after training
model.save("s2s_model.keras")


Number of samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21
Epoch 1/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.6895 - loss: 1.3212 - val_accuracy: 0.7309 - val_loss: 1.0845
Epoch 2/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7690 - loss: 0.8602 - val_accuracy: 0.7349 - val_loss: 1.0857
Epoch 3/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7991 - loss: 0.7248 - val_accuracy: 0.7464 - val_loss: 1.1141
Epoch 4/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8183 - loss: 0.6461 - val_accuracy: 0.7507 - val_loss: 1.1180
Epoch 5/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8311 - loss: 0.5862 - val_accuracy: 0.7598 - val_loss: 1.1073
Epoch 6/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8438 - loss: 0.5333 - val_accuracy: 0.7554 - val_loss: 1.0806
Epoch 7/180
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accur

In [23]:
import numpy as np
from keras.models import load_model, Model
from keras.layers import Input

# Load the trained seq2seq model
model = load_model("s2s_model.keras")

# Define encoder model
encoder_input = model.input[0]  # First input is encoder input
encoder_outputs, state_h, state_c = model.layers[2].output  # LSTM layer
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_input, encoder_states)

# Define decoder model
decoder_input = model.input[1]  # Second input is decoder input
latent_dim = state_h.shape[-1]  # Extract latent dimension

# Define new input layers for the decoder states
decoder_input_h = Input(shape=(latent_dim,), name="decoder_input_h")
decoder_input_c = Input(shape=(latent_dim,), name="decoder_input_c")
decoder_states_inputs = [decoder_input_h, decoder_input_c]

# Reuse LSTM and Dense layers from original model
decoder_lstm = model.layers[3]
decoder_dense = model.layers[4]

# Connect decoder layers
lstm_outputs, new_h, new_c = decoder_lstm(decoder_input, initial_state=decoder_states_inputs)
decoder_states = [new_h, new_c]
decoder_outputs = decoder_dense(lstm_outputs)

# Build final decoder model
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse dictionaries for decoding
reverse_input_char_index = {idx: char for char, idx in input_token_index.items()}
reverse_target_char_index = {idx: char for char, idx in target_token_index.items()}

# Decoder function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_text = ""
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_text += sampled_char

        if sampled_char == "\n":
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_text.strip()

# Example usage:
# transliteration = decode_sequence(input_seq)
# print("Devanagari transliteration:", transliteration)


In [24]:
import numpy as np
from keras.models import load_model, Model
from keras.layers import Input

# Load the trained seq2seq model
model = load_model("s2s_model.keras")

# Define encoder model
encoder_input = model.input[0]  # First input is encoder input
encoder_outputs, state_h, state_c = model.layers[2].output  # LSTM layer
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_input, encoder_states)

# Define decoder model
decoder_input = model.input[1]  # Second input is decoder input
latent_dim = state_h.shape[-1]  # Extract latent dimension

# Define new input layers for the decoder states
decoder_input_h = Input(shape=(latent_dim,), name="decoder_input_h")
decoder_input_c = Input(shape=(latent_dim,), name="decoder_input_c")
decoder_states_inputs = [decoder_input_h, decoder_input_c]

# Reuse LSTM and Dense layers from original model
decoder_lstm = model.layers[3]
decoder_dense = model.layers[4]

# Connect decoder layers
lstm_outputs, new_h, new_c = decoder_lstm(decoder_input, initial_state=decoder_states_inputs)
decoder_states = [new_h, new_c]
decoder_outputs = decoder_dense(lstm_outputs)

# Build final decoder model
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse dictionaries for decoding
reverse_input_char_index = {idx: char for char, idx in input_token_index.items()}
reverse_target_char_index = {idx: char for char, idx in target_token_index.items()}

# Decoder function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_text = ""
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_text += sampled_char

        if sampled_char == "\n":
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_text.strip()

# Example usage:
# transliteration = decode_sequence(input_seq)
# print("Devanagari transliteration:", transliteration)


In [26]:
import numpy as np

# Define constants based on your training configuration
max_encoder_seq_length = 15
max_decoder_seq_length = 20

# Ensure these are already loaded before this script is run:
# input_token_index, target_token_index, reverse_target_char_index, encoder_model, decoder_model

num_encoder_tokens = encoder_model.input_shape[-1]  # Get the last dimension of the input shape
num_decoder_tokens = len(target_token_index)

# Function to encode input text into one-hot encoded format for the encoder
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
        else:
            print(f"Warning: character '{char}' not in training vocabulary.")
    for t in range(len(input_text), max_encoder_seq_length):
        encoder_input[0, t, input_token_index[' ']] = 1.0
    return encoder_input

# Function to decode the predicted sequence from the decoder
def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_sentence = ""
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence.strip()

# Command-line interface to test transliteration interactively
def translate_latin_to_devanagari():
    while True:
        user_input = input("Latin word: ").strip()
        if user_input.lower() == "stop":
            break

        input_seq = encode_input_text(user_input)
        output = decode_sequence(input_seq)
        print("Devanagari word:", output)

# Start interactive mode
translate_latin_to_devanagari()


Latin word: sindhu
Devanagari word: किसंडह
Latin word: stop
